# The Battle of Neighborhoods

## Getting the longitude and latitude data

    Let's define China town as the center of the city. 
   

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [1]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
address = 'Chinatown, San Francisco, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.7943011 -122.4063757


We will then generate the list of neighborhoods candidate around the Chinatown in the city. We use the information on this website: http://www.healthysf.org/bdi/outcomes/zipmap.htm
 

In [10]:
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])
df.columns = df.iloc[0]
df = df.iloc[1:-1, :-1]
df=df.rename(columns={'Zip Code': 'Zipcode'})
sf_data = df
print(sf_data.head)

<bound method NDFrame.head of 0  Zipcode                             Neighborhood
1    94102  Hayes Valley/Tenderloin/North of Market
2    94103                          South of Market
3    94107                             Potrero Hill
4    94108                                Chinatown
5    94109             Polk/Russian Hill (Nob Hill)
6    94110             Inner Mission/Bernal Heights
7    94112       Ingelside-Excelsior/Crocker-Amazon
8    94114                        Castro/Noe Valley
9    94115               Western Addition/Japantown
10   94116                     Parkside/Forest Hill
11   94117                           Haight-Ashbury
12   94118                           Inner Richmond
13   94121                           Outer Richmond
14   94122                                   Sunset
15   94123                                   Marina
16   94124                    Bayview-Hunters Point
17   94127    St. Francis Wood/Miraloma/West Portal
18   94131                     Twi

In order to get the longitude and latitude data information, we will use uszip to search the information with zipcode

In [12]:
!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in sf_data.iterrows():
    zipcode = search.by_zipcode(row["Zipcode"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

sf_data["Latitude"] = latitude
sf_data["Longitude"] = longitude

sf_data.head()

,Zipcode,Neighborhood,Latitude,Longitude
1,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420
2,94103,South of Market,37.780,-122.410
3,94107,Potrero Hill,37.770,-122.390
4,94108,Chinatown,37.791,-122.409
5,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420


Let's do a map to visualize the neighborhoods using folium

In [19]:
sf_map = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(sf_data['Latitude'], sf_data['Longitude'], sf_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(sf_map)  
    
sf_map

In [18]:
from IPython.display import Image 
from IPython.core.display import HTML 

# Foursquare Restaurant Data

## Define Foursquare Credentials

In [105]:
CLIENT_ID = 'THDIJDVKVDJ2V4Q44DA2PNQSZXZZHYVIMBVGRIQ2V5UB2PPH' # your Foursquare ID
CLIENT_SECRET = 'FKQOYGLEDHQWBGCB5DO3KEDAHGDMF2NCOMCCHTJD0RARXQ2Q' # your Foursquare Secret
ACCESS_TOKEN = '4QAUBOWNAQZX2XC3PYNU4LR45J0TLENZVQC3IPIX5TNXKPCM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: THDIJDVKVDJ2V4Q44DA2PNQSZXZZHYVIMBVGRIQ2V5UB2PPH
CLIENT_SECRET:FKQOYGLEDHQWBGCB5DO3KEDAHGDMF2NCOMCCHTJD0RARXQ2Q


 ## Get the Chinese Restaurants in each neighborhoods

In [113]:
# root categories for Asian Restaurant
ch_food_category='4bf58dd8d48988d145941735'
radius=1000
chinese_restaurant_categories=['52af3a5e3cf9994f4e043bea','52af3a723cf9994f4e043bec',
                   '52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3',
                   '52af3a673cf9994f4e043beb','52af3a903cf9994f4e043bee',
                   '4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef',
                   '52af3aaa3cf9994f4e043bf0','52af3ab53cf9994f4e043bf1',
                   '52af3abe3cf9994f4e043bf2','52af3ac83cf9994f4e043bf3',
                   '52af3ad23cf9994f4e043bf4','52af3add3cf9994f4e043bf5',
                   '52af3af23cf9994f4e043bf7','52af3ae63cf9994f4e043bf6',
                   '52af3afc3cf9994f4e043bf8','52af3b053cf9994f4e043bf9',
                   '52af3b213cf9994f4e043bfa','52af3b293cf9994f4e043bfb',
                   '52af3b343cf9994f4e043bfc','52af3b3b3cf9994f4e043bfd',
                   '52af3b463cf9994f4e043bfe','52af3b633cf9994f4e043c01',
                   '52af3b513cf9994f4e043bff','52af3b593cf9994f4e043c00',
                   '52af3b6e3cf9994f4e043c02','52af3b773cf9994f4e043c03',
                   '52af3b813cf9994f4e043c04','52af3b893cf9994f4e043c05',
                   '52af3b913cf9994f4e043c06','52af3b9a3cf9994f4e043c07',
                   '52af3ba23cf9994f4e043c08']

In [122]:
def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'lunch', 'food']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific


In [130]:
def get_restaurants(name,lats, lons):
    venues_list = []
 
    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, ch_food_category, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   item['venue']['location'],
                   item['venue']['location']['distance']) for item in results]  
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter=chinese_restaurant_categories)
            if is_res:
                if is_chinese:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        venue_name, 
                        venue_categories, 
                        venue_latlon, 
                        venue_address, 
                        venue_distance)])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 'Venue Category',
                  'Venue Latitude&Longitude', 
                  'Venue address', 'Venue Distance' 
                  ]
    return(nearby_venues)

In [131]:

chinese_restaurants= get_restaurants(sf_data['Neighborhood'],sf_data['Latitude'], sf_data['Longitude'])

Obtaining venues around candidate locations:

In [132]:
chinese_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude&Longitude,Venue address,Venue Distance
0,1 Hayes Valley/Tenderloin/North of Market ...,37.780,-122.41,Wing Lum,"[(Dim Sum Restaurant, 4bf58dd8d48988d1f5931735)]","(37.7874238737051, -122.42007375704523)","{'address': '1150 Polk St', 'crossStreet': 'Su...",826
1,1 Hayes Valley/Tenderloin/North of Market ...,37.780,-122.41,Sizzling Pot King,"[(Szechuan Restaurant, 52af3b773cf9994f4e043c03)]","(37.77711002625363, -122.4125721848816)","{'address': '139 8th St', 'lat': 37.7771100262...",728
2,1 Hayes Valley/Tenderloin/North of Market ...,37.780,-122.41,Sizzling Pot King,"[(Szechuan Restaurant, 52af3b773cf9994f4e043c03)]","(37.77711002625363, -122.4125721848816)","{'address': '139 8th St', 'lat': 37.7771100262...",393
3,1 Hayes Valley/Tenderloin/North of Market ...,37.780,-122.41,Henry's Hunan Restaurant,"[(Hunan Restaurant, 52af3afc3cf9994f4e043bf8)]","(37.77217872722743, -122.4075159398087)","{'address': '1016 Bryant St', 'crossStreet': '...",897
4,1 Hayes Valley/Tenderloin/North of Market ...,37.791,-122.41,R&G Lounge 嶺南小館,"[(Cantonese Restaurant, 52af3a7c3cf9994f4e043b...","(37.79407153466883, -122.40472405865627)","{'address': '631 Kearny St', 'crossStreet': 'a...",508
5,1 Hayes Valley/Tenderloin/North of Market ...,37.791,-122.41,Z & Y Restaurant,"[(Szechuan Restaurant, 52af3b773cf9994f4e043c03)]","(37.79611214947366, -122.40607799181394)","{'address': '655 Jackson St', 'crossStreet': '...",624
6,1 Hayes Valley/Tenderloin/North of Market ...,37.791,-122.41,Chong Qing Xiao Mian,"[(Szechuan Restaurant, 52af3b773cf9994f4e043c03)]","(37.7964178, -122.4053695)","{'address': '915 Kearny St', 'crossStreet': 'C...",682
7,1 Hayes Valley/Tenderloin/North of Market ...,37.791,-122.41,Wing Sing Dim Sum,"[(Dim Sum Restaurant, 4bf58dd8d48988d1f5931735)]","(37.79627755329796, -122.40832679270122)","{'address': '1125 Stockton St', 'crossStreet':...",590
8,1 Hayes Valley/Tenderloin/North of Market ...,37.791,-122.41,Dim Sum Corner,"[(Dim Sum Restaurant, 4bf58dd8d48988d1f5931735)]","(37.792627, -122.406181)","{'address': '601 Grant Ave', 'crossStreet': 'C...",307
9,1 Hayes Valley/Tenderloin/North of Market ...,37.790,-122.41,Wing Lum,"[(Dim Sum Restaurant, 4bf58dd8d48988d1f5931735)]","(37.7874238737051, -122.42007375704523)","{'address': '1150 Polk St', 'crossStreet': 'Su...",286
